# QA Baseline without RAG

This notebook is a baseline for the QA task without the RAG model. For a fair comparison, we choose the same backbone model as the one in the RAG pipeline: the `meta/llama3.1-8b-Instruct` model. We also adopt the same data type (fp16) and the same config for setting up the tokenizer. We use the same prompt format as the one in the RAG pipeline.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

generation_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer, 
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Step 3: load qa annotation test set
import pandas as pd

qa_df = pd.read_csv("../data/crawled_text_datas/QA_pairs_1.csv")

# doc_ids = qa_df["Doc_id"].tolist()
questions = qa_df["Question"].tolist()


In [ ]:
template = """
Bạn là một trợ lý chuyên gia trả lời các câu hỏi thực tế về nhiều khía cạnh của Đại học quốc gia Hà Nội, bao gồm lịch sử, chính sách, văn hóa, sự kiện và nhiều hơn nữa.
Nếu bạn không biết câu trả lời, hãy nói "Tôi không biết."

Hướng dẫn quan trọng:
- Trả lời ngắn gọn, không lặp lại câu hỏi.
- KHÔNG dùng câu hoàn chỉnh. Chỉ cung cấp từ, tên, ngày tháng hoặc cụm từ trực tiếp trả lời câu hỏi. Ví dụ, với câu hỏi "Đại học Quốc gia Hà Nội được thành lập năm nào?", bạn chỉ trả lời "1993".

Ví dụ:
Q: Đại học Quốc Gia Hà Nội được thành lập năm nào? A: 1993
Q: Hiệu trưởng đầu tiên của trường đại học công nghệ là ai? A: Nguyễn Văn Hiệu

Câu hỏi: {question} \n\n

Trả lời:
"""


In [4]:
# use the template the generate the answers
from tqdm import tqdm
generated_answers = []
for question in tqdm(questions):
    full_prompt = template.format(question=question)
    messages = [
        {"role": "user", "content": full_prompt},
        ]
    output = generation_pipe(messages, max_new_tokens=50)
    generated_answers.append(output[0]["generated_text"][1]['content'])

100%|██████████| 574/574 [02:38<00:00,  3.61it/s]


In [ ]:
# write all columns to a csv file
# results_df = pd.DataFrame({
#         "Doc_id": doc_ids,
#         "Question": questions,
#         "Reference_Answers": answers,
#         "Generated_Answer": generated_answers,
#     })

results_df = pd.DataFrame({
        "Question": questions,
        "Generated_Answer": generated_answers,
    })

# save the results to a csv file
results_df.to_csv("../output/baseline_no_rag.csv", index=False)